In [16]:
! pip install torch
! pip install numpy

In [17]:
import torch
import numpy as np
from itertools import pairwise

## Motive

Implement a general neural net that does not require anything more than np.

In [216]:
# For loss function:
def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

def Sigmoid_derivative(x):
        return x * (1 - x)


class NeuralNetwork:

    """
    A more 'traditional' implementation of a neural network, with manually loaded layers.
    """

    def __init__(self, input_size, hidden_size, output_size,
                 input_weights=None, hidden_weights=None, 
                 input_bias=None, hidden_bias=None, output_bias=None):

        """
        Should store:
            - array of inputs H_{t}
            - array of weights W_{t} (going towards H_{t+1})
        """

        self.loss_value = 0

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize with random weights.
        self.input_weights = input_weights if input_weights is not None else np.random.rand(self.input_size, self.hidden_size)
        self.hidden_weights = hidden_weights if hidden_weights is not None else np.random.rand(self.hidden_size,output_size)
        
        self.input_bias = input_bias #if input_bias is not None else np.random.rand(num_points, self.input_size)
        self.hidden_bias = hidden_bias #if hidden_bias is not None else np.random.rand(num_points, self.hidden_size)
        self.output_bias = output_bias #if output_bias is not None else np.random(num_points, self.output_size)

    def show_neural_network(self) -> None:
        print(self.input_weights)
        print(self.hidden_weights)

    def activate(self, x):
        return Sigmoid(x)
    
    def deriv_act(self, x):
        return Sigmoid_derivative(x)
    
    def loss(self, y_true, y_calc):
        return np.divide(np.sum(np.square(y_true - y_calc)), 2)

    def deriv_loss(self, y_true, y_calc):
        return y_calc - y_true

    def forward(self, data) -> None:
        # Update every bias.
        self.input_bias = data
        hidden_z = np.matmul(self.input_weights, self.input_bias)
        #print("hidden preac:\n", hidden_z)
        self.hidden_bias = self.activate(hidden_z)
        output_z = np.matmul(self.hidden_weights, self.hidden_bias)
        #print("output preac:\n", output_z)
        self.output_bias = self.activate(output_z)
        return self.output_bias
        
    def backward(self, y_true, learning_rate):

        # For T in T -> 1:
        # Derive d(L, Wi), d(L, H{i-1})

        #print(np.array([self.deriv_act(i) for i in self.output_bias])) #* self.deriv_loss(y_true, self.output_bias)
        #print(self.deriv_loss(y_true, self.output_bias))
        dsim_output = np.array([self.deriv_act(i) for i in self.output_bias]) * self.deriv_loss(y_true, self.output_bias)
        #print(dsim_output)
        #print(self.hidden_bias.shape)
        dsim_hidden_weights = np.matmul(dsim_output, np.transpose(self.hidden_bias))
        dsim_hidden_bias = np.matmul(np.transpose(self.hidden_weights), dsim_output)
        #print(dsim_hidden_weights)
        #print(dsim_hidden_bias)

        dsim_hidden = np.array([self.deriv_act(i) for i in self.hidden_bias]) * dsim_hidden_bias
        dsim_input_weights = np.matmul(dsim_hidden, np.transpose(self.input_bias))
        dsim_input_bias = np.matmul(np.transpose(self.input_weights), dsim_hidden)

        #print(dsim_input_weights)
        #print(dsim_input_bias)
        
        self.hidden_weights -= learning_rate * dsim_hidden_weights
        self.input_weights -= learning_rate * dsim_input_weights

        return 0
    

    def train_neural_network(self, train_x, train_y, learning_rate = 0.01, num_epochs=1):

        """
        Trains the neural network on a dataset.
        """
        result = 0
        for epochs in range(num_epochs):
            fore = self.forward(train_x)
            result = fore
            self.backward(learning_rate=learning_rate, y_true=train_y)
            self.loss_value = self.loss(fore, train_y)
            print(self.loss_value)

        return result


In [220]:
def run_trial(train_x, train_y, 
              input_size, hidden_size, output_size,
              input_weights, hidden_weights,num_epochs=10):


    trial = NeuralNetwork(input_size=input_size, hidden_size=hidden_size, output_size=output_size, 
                        input_weights=input_weights,
                        hidden_weights=hidden_weights)
    trial.show_neural_network()
    res = trial.train_neural_network(train_x=train_x, train_y=train_y,num_epochs=num_epochs)
    trial.show_neural_network()
    return res

In [143]:
run_trial(train_x= np.array([[1],
                             [1]]),
          train_y = np.array([[1]]),
          input_size = 2, hidden_size= 2, output_size=2,
          input_weights=np.array([[0.1,-.4],[-0.1,-0.1]]),
          hidden_weights=np.array([[0.06,-0.4]])
          )

[[ 0.1 -0.4]
 [-0.1 -0.1]]
[[ 0.06 -0.4 ]]
[[0.46144346]]
0.1450215731924639


In [134]:
run_trial(train_x= np.array([[1],
                             [9]]),
          train_y = np.array([[4]]),
          input_size = 2, hidden_size= 2, output_size=2,
          input_weights=np.array([[0.1,-.4],
                                  [-0.1,-0.1]]),
          hidden_weights=np.array([[0.06,-0.4]])
          )

[[ 0.1 -0.4]
 [-0.1 -0.1]]
[[ 0.06 -0.4 ]]
[[0.4735702]]
[[-0.02576968 -0.23643827]]
[[-0.00150086 -0.01350773]
 [ 0.06914009  0.62226081]]
6.217853571247387


In [135]:
run_trial(train_x= np.array([[1,1],
                             [1,9]]),
          train_y = np.array([[1,4]]),
          input_size = 2, hidden_size= 2, output_size=2,
          input_weights=np.array([[0.1,-.4],
                                  [-0.1,-0.1]]),
          hidden_weights=np.array([[0.06,-0.4]])
          )

[[ 0.1 -0.4]
 [-0.1 -0.1]]
[[ 0.06 -0.4 ]]
[[0.46144346 0.4735702 ]]
[[-0.08272566 -0.29668782]]
[[-0.00346393 -0.0154708 ]
 [ 0.08239099  0.63551171]]
6.362875144439851


In [130]:
run_trial(train_x = np.array([[1,0],[1,9],[2,-9]]),
          train_y = np.array([[1, 0]]),
          input_size=3, hidden_size=2, output_size=1, 
          input_weights=np.array([[0.1, 0.4, 0.3],[-0.1,-0.1, 0.5]]),
          hidden_weights=np.array([[0.06, -0.4]]))

[[ 0.1  0.4  0.3]
 [-0.1 -0.1  0.5]]
[[ 0.06 -0.4 ]]
[[0.44251181 0.51021319]]
0.285555295658419


In [ ]:
a = np.array([[-0.00196308, -0.00196308],
 [ 0.0132509,   0.0132509 ]])
b = np.array([[-0.00150086, -0.01350773],
 [ 0.06914009,  0.62226081]])


c = np.array([[-0.00346393, -0.0154708 ],
 [ 0.08239099,  0.63551171]])

a + b - c

## Generate Data

Now we generate some data to check if the generalized case works properly.

In [170]:
from scipy.stats import multivariate_normal

In [ ]:
num_data = 1000
np.random.seed(42)

In [204]:
mv1 = multivariate_normal(mean=np.array([3,2,1]),
                    cov=np.array([[1,0.5,0.51],
                                [0.5,1,0.49],
                                [0.51,0.49,1]]),allow_singular=True,seed=42).rvs(size=1000)
mv2 = multivariate_normal(mean=np.array([8,1,4]),
                    cov=np.array([[1,0.5,0.51],
                                [0.5,1,0.49],
                                [0.51,0.49,1]]),allow_singular=True,seed=88).rvs(size=1000)

print(mv1.shape, mv2.shape)
mv1_true = np.zeros(1000)
mv2_true = np.ones(1000)
mv = np.concatenate([mv1, mv2], axis=0)
mv_true = np.concatenate([mv1_true, mv2_true],axis=0)

(1000, 3) (1000, 3)


In [222]:
final_run = run_trial(train_x = mv.T,
          train_y = mv_true,
          input_size=3, hidden_size=2, output_size=1, 
          input_weights=np.array([[0.1, 0.4, 0.3],[-0.1,-0.1, 0.5]]),
          hidden_weights=np.array([[0.06, -0.4]]),
          num_epochs=10000)

[[ 0.1  0.4  0.3]
 [-0.1 -0.1  0.5]]
[[ 0.06 -0.4 ]]
263.7692813343932
252.4809261153115
243.1159610128714
235.3361903867314
220.1385294781332
191.3582801986322
232.2378060213453
320.3579292628009
216.9630825149029
220.84097150801244
266.88353964412215
210.05214756418536
185.86435304666955
181.81637202390064
203.85411543095393
245.18714417575842
188.19119350018613
192.36247936044649
172.3470534303678
174.10772955371274
159.62280826553257
154.88429318055321
128.07258224194493
85.33045964644305
57.70701396031548
65.42543733825066
54.50001925848798
77.67192485359178
47.67184048084501
57.48974073675825
44.43596533518639
53.13713116620676
42.2928532384885
49.73630160075945
40.69104277604701
47.282850154600226
39.45347069149068
45.46628854219154
38.465791353473904
44.06930921429864
37.65712367103819
42.96370750295254
36.981530526179014
42.068482147554064
36.407697149468774
41.32923319273224
35.91340203199557
40.707906616375865
35.482349465318535
40.177045528013
35.102254292162215
39.71633431

In [233]:
count = 0
for i in range(len(final_run[0])):
    if i >= 1000 and final_run[0][i] < 0.5:
        count += 1
    elif i < 1000 and final_run[0][i] >= 0.5:
        count += 1
print(count)

61
